In [ ]:
import flopy
import os
import geopandas as gpd

import contextily as ctx
import pandas as pd
import numpy as np
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import conda_scripts.wiski as wiski
import pathlib
import basic
import matplotlib.dates as mdates
import conda_scripts.gwplot_fancy as gwp
import conda_scripts 

In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2016')

datestart = info['start_date']

name = info['name']

out_folder = basic.out_folder('June2016')

basic
print(datestart)
print(out_folder)

In [ ]:


from flopy.utils.postprocessing import (
    get_transmissivities,
    get_water_table,
    get_gradients,
)
import flopy.utils.binaryfile as bf

print(sys.version)
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(mpl.__version__))
print("flopy version: {}".format(flopy.__version__))

In [ ]:
ml = basic.load_model()

In [ ]:
import conda_scripts.utils as utils

In [ ]:
ibound = gpd.read_file("GIS/iboundlay1.shp")
ibound = ibound.query("ibound_1==1")
ibound.head(1)

swr = gpd.read_file("GIS/SWR_Reaches.shp")

sfr = gpd.read_file('SFR_files/only_sfr_cells.shp')

In [ ]:
def get_well_locations():
    p = r"T:\smaples\USGS-LBNL_WQ\Wohler_MW_inventory\Monitoring Wells Site Visit Notes - Wohler+Mirabel_car.xlsx"
    
    t = pd.read_excel(p)
    t = t.loc[t.loc[:,'Longitude (iphone gps)'].notnull(),:]
    df = gpd.GeoDataFrame(t, geometry = gpd.points_from_xy(t.loc[:,'Longitude (iphone gps)'], t.loc[:,'Latitude (iphone gps)'], crs = 4326))
    
    df = df.to_crs(2226)
    
    return df

def get_wiski():
    # gw = wiski.wiski.get_gw_stations_in_basin(basins= ['LRR*'], final_only = False)

    meta = wiski.wiski.get_gw_stations_wellmeta_in_basin(basins = ['LRR*'])
    meta = meta.dropna(subset  = 'station_name')
    meta = gpd.GeoDataFrame(meta, geometry = gpd.points_from_xy(meta.station_longitude, meta.station_latitude), crs  = 4326).to_crs(2226)
    
    return  meta



In [ ]:
wiski_meta = get_wiski()

wells = get_well_locations()



In [ ]:
wells_mod = pd.merge(wiski_meta, wells.loc[:,['Well Name', 'WISKI','Notes_SX', 'Notes_SRM', 
                  'USGS Map ID (https://pubs.usgs.gov/ds/610/pdf/ds610.pdf)','USGS NWIS ID',
                 'WCR (Y/N)','Total completed depth (ft bgs)', 'Total depth (ft bgs)',
                'Screened interval (ft bgs)', 'casing diameter (inches)',]] ,
                     left_on = 'station_name', right_on = 'WISKI', how = 'inner')

wells_mod = gpd.GeoDataFrame(wells_mod, geometry = gpd.points_from_xy(wells_mod.station_longitude, wells_mod.station_latitude), crs  = 4326).to_crs(2226)


In [ ]:
import importlib

In [ ]:
importlib.reload(conda_scripts )

In [ ]:
# mm = conda_scripts.gwplot_fancy.fancy_plot(row['Station name'])

In [ ]:
# mm = conda_scripts.gwplot_fancy.fancy_plot('LRR0091')

# mm.do_plot(close_plot=False)

In [ ]:
# mm.do_plot()

In [ ]:
conda_scripts.arich_functions.add_basemaps()

In [ ]:
mm.t.gw_elev

In [ ]:
mm = conda_scripts.gwplot_fancy.fancy_plot('LRR0146')

# mm.do_plot(close_plot=False, skip_gw_data= False,seasonal = False, remove_pt = True, hydro_plot_xlims = [2010, None],
#            maptype =  'USA_Topo_Maps',y_axis_range_min = 80,
#            map_buffer_size=6000)
mm.do_plot(close_plot=False, skip_gw_data= False,seasonal = False, remove_pt = True, hydro_plot_xlims = [2010, None],
           maptype =  'ctx.Esri.WorldImagery', y_axis_range_min = 80,
           map_buffer_size=4000)

In [ ]:
colors[1]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

lwbase = plt.rcParams['lines.linewidth']
thin = lwbase / 2
thick = lwbase * 3

fig, axs = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True)
for icol in range(2):
    if icol == 0:
        lwx, lwy = thin, lwbase
    else:
        lwx, lwy = lwbase, thick
    for irow in range(2):
        for i, color in enumerate(colors):
            axs[irow, icol].axhline(i, color=color, lw=lwx)
            axs[irow, icol].axvline(i, color=color, lw=lwy)

    axs[1, icol].set_facecolor('k')
    axs[1, icol].xaxis.set_ticks(np.arange(0, 10, 2))
    axs[0, icol].set_title(f'line widths (pts): {lwx:g}, {lwy:g}',
                           fontsize='medium')

for irow in range(2):
    axs[irow, 0].yaxis.set_ticks(np.arange(0, 10, 2))

fig.suptitle('Colors in the default prop_cycle', fontsize='large')

plt.show()

In [ ]:
k = conda_scripts.wiski.wiski.get_kiwis()

In [ ]:
','.join(missing.loc[:,'Station name'].values)

In [ ]:
ts = k.get_timeseries_list(station_name = ','.join(missing.loc[:,'Station name'].values),
return_fields = ['station_name','ts_id','ts_name','coverage'])

In [ ]:
not_in_model = ts.loc[pd.to_datetime(ts.to).notnull()]

In [ ]:
import geemap

In [ ]:
geemap.geoda

In [ ]:
gw = k.get_station_list(station_name ='LRR0*')
gw = gw.loc[gw.station_name.isin(not_in_model.station_name)]

gw = gpd.GeoDataFrame(gw, geometry = gpd.points_from_xy(gw.station_longitude, gw.station_latitude), crs = 4326)

Map = geemap.Map(center=( 38.517, -122.882), zoom=14)
Map.add_data(gw, 'station_name')
Map.add_labels(geemap.geopandas_to_ee(gw),'station_name')
Map.save('missing_wells.html')


In [ ]:


','.join(missing.loc[:,'Station name'].values)

ts = k.get_timeseries_list(station_name = ','.join(missing.loc[:,'Station name'].values),
return_fields = ['station_name','ts_id','ts_name','coverage'])



# here are all of the wells in mirabel wohler with any data

In [ ]:
f = "Waterlevel_Data/Pressure_data_editing_2023/wiski_mirable_wohler_stations.csv"

f = pathlib.Path(f)

wisk = pd.read_csv(f)
k = conda_scripts.wiski.wiski.get_kiwis()
ts = k.get_timeseries_list(station_name = ','.join(wisk.loc[:,'Station name'].values),
return_fields = ['station_name','ts_id','ts_name','coverage'])


mirabel_wohler_wells = ts.loc[pd.to_datetime(ts.loc[:,'from']).notnull()].drop_duplicates('station_name')

In [ ]:
ts = k.get_timeseries_list(station_name = ','.join(wisk.loc[:,'Station name'].values),
return_fields = ['station_name','ts_id','ts_name','coverage'])

In [ ]:
ts.query("ts_name=='04.Calculated Head Offset Edited'")

In [ ]:
ts.ts_name.unique()

In [ ]:
mm.t.gw_elev

In [ ]:
conda_scripts.arich_functions.add_basemaps(None, 'ctx', True)

# these are all of the stations found in wiski in the geographic area

In [ ]:

all_meas.set_index('Station_Name', append = True).drop(columns = 'Source').groupby(level = [0,1]).mean().unstack().resample('1D').mean().droplevel(0,1).plot()


In [ ]:
f = "Waterlevel_Data/Pressure_data_editing_2023/wiski_mirable_wohler_stations.csv"

f = pathlib.Path(f)


In [ ]:
caissons = pd.DataFrame([f"LRR0{n}" for n in range(225,231)], columns = ['station_name'])
all_wells = pd.concat([mirabel_wohler_wells,caissons])
all_wells.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
all_meas.drop(columns = 1)

In [ ]:
all_meas.drop(columns = 1).set_index(['Station_Name','Source'], append = True).groupby([
                                                                                       pd.Grouper(level = 1),
    pd.Grouper(freq = '1D', level = 0),
                                                                                       pd.Grouper(level = 2)]).mean().unstack([0,2]).droplevel(0,1)

In [ ]:
all_meas.to_csv('C:\modeling\MirabelWohler\Waterlevel_Data\mirabel_wohler_allGW_meas.csv', index = True)

In [ ]:
all_meas = pd.DataFrame()
cnt = 0
# for _,row in mirabel_wohler_wells.iterrows():
for _,row in all_wells.iterrows():

    filename = f.parent.joinpath(row['station_name']+'.png' )
    print(filename)
    
    mm = conda_scripts.gwplot_fancy.fancy_plot(row['station_name'], filename = filename)
    

    mm.do_plot(close_plot=True, skip_gw_data= False,seasonal = False, 
               remove_pt = True, hydro_plot_xlims = [2010, None],
           maptype =  'ctx.Esri.WorldImagery', y_axis_range_min = 80,
           map_buffer_size=4000, plot_nearby=False)
    
    pres = pd.DataFrame(columns = [1])
    man = pd.DataFrame(columns = [1])
    
    if mm.t.pressure_trans is not None:
        pres = mm.t.pressure_trans
        pres.columns = ['Value']
        pres.loc[:,'Station_Name'] = row['station_name']
        pres.loc[:,'Source'] = 'Pressure Transducer'

    
    if mm.t.manual_meas is not None:
        man = mm.t.manual_meas
        man.columns = ['Value']
        man.loc[:,'Station_Name'] = row['station_name']
        man.loc[:,'Source'] = 'Manual Measurement'
    
    all_meas = pd.concat([all_meas, man, pres])
    

out_df = all_meas.drop(columns = 1).set_index(['Station_Name','Source'], append = True).groupby([
                                                                                       pd.Grouper(level = 1),
    pd.Grouper(freq = '1D', level = 0),
                                                                                       pd.Grouper(level = 2)]).mean().unstack([0,2]).droplevel(0,1)
out_df = out_df.sort_index()

out_df.to_csv('C:\modeling\MirabelWohler\Waterlevel_Data\mirabel_wohler_allGW_meas.csv', index = True)

In [ ]:
import seaborn as sns

In [ ]:
lon = k.get_station_list(station_name = 'LRR*', return_fields = [ 'station_name','station_longitude','station_latitude','station_no'])

In [ ]:
out_df.columns.get_level_values(0)

In [ ]:
from conda_scripts import arich_functions as af

In [ ]:
mmkp = conda_scripts.make_map.make_map("Locs")
mmkp.plotloc()

In [ ]:
stat = gpd.GeoDataFrame(lon, geometry = gpd.points_from_xy(lon.station_longitude, lon.station_latitude),crs= 4326)
stat = stat.loc[stat.station_name.isin(out_df.columns.get_level_values(0))]



In [ ]:
daily = out_df.stack([0,1]).groupby(level = [0,1]).mean().unstack().T

daily = pd.merge(daily, lon, left_index = True, right_on = 'station_name')
daily = daily.sort_values('station_longitude').drop(columns = ['station_longitude','station_name','station_latitude'])
daily = daily.set_index('station_no')

daily = daily.drop('Caisson1')
fig, ax = plt.subplots(figsize= (20,20))

daily =daily.T
daily.index = pd.to_datetime(daily.index)
daily.index = daily.index.strftime('%b-%Y')

daily = daily.T

daily = daily.loc[daily.count(axis=1)>365,:]
sns.heatmap(daily, ax = ax,cmap = 'gist_ncar',vmax = 70, vmin = -30)
ax.set_facecolor('grey')
# plt.savefig('Waterlevel_Data/all_wl.png', bbox_inches= 'tight')
# import matplotlib.dates as mdates
# myFmt = mdates.DateFormatter('%b-%Y')
# ax.xaxis.set_major_formatter(myFmt)
# ax.xaxis.set_major_locator(mdates.)
# ax.set_xticklabels(daily.index.strftime('%d-%m-%Y'))

In [ ]:
dfall = pd.read_csv('C:\modeling\MirabelWohler\Waterlevel_Data\mirabel_wohler_allGW_meas.csv',header = [0,1],index_col=[0],parse_dates=True)
dfall.stack().groupby(level = 0).mean()

In [ ]:
# # these are all of the stations found in wiski in the geographic area



# f = pathlib.Path(f)

# wisk = pd.read_csv(f)


# # missing = wells_mod.loc[~wells_mod.station_name.isin(wisk.loc[:,'Station name'])]

# missing = wisk.loc[~wisk.loc[:,'Station name'].isin(wells_mod.station_name)]


# for _,row in missing.iterrows():
#     filename = f.parent.joinpath(row['Station name']+'.png' )
#     print(filename)
#     mm = conda_scripts.gwplot_fancy.fancy_plot(row['Station name'], filename = filename)
#     mm.do_plot(close_plot=True,skip_gw_data = False, remove_pt = True, seasonal = False, map_buffer_size = 2000)
    
#     # mm.t.get_station_pars()
#     # mm.t.check_if_no_gw_data()
#     # if mm.t.gw_data_empty:
#     #     # if mm.t.gw_elev.shape[0]==0:
#     #     #     print(f"skipping {row['Station name']}")
#     #     mm.do_plot(close_plot=False,skip_gw_data = True)
#     # else:
#     #     mm.do_plot(close_plot=False,skip_gw_data = False)
#     #     # try:
#     #     #     mm.do_plot(close_plot=False,skip_gw_data = False)
#     #     # except:
#     #     #     print(f"failed {row['Station name']}")

# these are the caissons

In [ ]:
import conda_scripts.gwplot_wiski as gw_plot

In [ ]:
k = conda_scripts.wiski.wiski.get_kiwis()

In [ ]:
mm.t.pressure_trans

In [ ]:
pd.DataFrame(np.random.random((10,10))).plot(style = 's',color = 'r', markerfacecolor = 'g')

In [ ]:
caissons = pd.DataFrame([f"LRR0{n}" for n in range(225,231)], columns = ['station_name'])

In [ ]:
for _,row in caissons.sort_values('station_name',ascending = False).iterrows():
    filename = f.parent.joinpath(row['station_name']+'.png' )
    print(filename)
    
    gw_elev = gw_plot.wiski_plot(row['station_name'])
    gw_elev.get_station_pars()
    gw_elev = gw_elev.gw_elev
    gw_elev = gw_elev.query("ts_name=='03.FinalElevation'")
    print(gw_elev)

    mm = conda_scripts.gwplot_fancy.fancy_plot(row['station_name'], filename = filename,group=gw_elev)
    

    mm.do_plot(close_plot=True, skip_gw_data= False,seasonal = False, remove_pt = True, hydro_plot_xlims = [2010, None],
           maptype =  'ctx.Esri.WorldImagery', y_axis_range_min = 80,
           map_buffer_size=4000, plot_nearby=False)


In [ ]:
[ymin, ymax] == None

In [ ]:
ymin = 110
ymax = 151
ymin = np.floor(ymin/10)*10
ymax = np.ceil(ymax / 10) * 10

print(ymin, ymax)

In [ ]:
meta

In [ ]:
wells_mod.to_html(os.path.join(out_folder,'observation_wells.html'))

In [ ]:


grid = flopy.utils.gridintersect.GridIntersect(ml.modelgrid)

inx = [grid.intersect(x, shapetype = 'point').cellids for x in wells_mod.geometry.tolist() ]
r = [x[0][0] if len(x)>0 else np.nan for x in inx]
c = [x[0][1] if len(x)>0 else np.nan for x in inx]
inside = [True if len(x)>0 else False for x in inx]

wells_mod.loc[:,'i_r'] = r
wells_mod.loc[:,'j_c'] = c
wells_mod.loc[:,'inmodearea'] = inside

wells_mod.columns

In [ ]:
x = """<div class="gallery">
  <a target="_blank" href="hydrographs/{:}">
    <img src="hydrographs/{:}" alt="{:}" width="600" height="400">
  </a>
  <div class="desc">{:}</div>
</div>""".format
f = os.listdir(os.path.join(out_folder,'hydrographs'))

[print(x(i, i, i.strip('.png'),  i.strip('.png'))) for i in f]

In [ ]:
wells_mod.to_file("GIS/wells_mod.geojson")
wells_mod.to_file("GIS/wells_mod.shp")

In [ ]:
gpd.read_file("GIS/wells_mod.geojson")

In [ ]:
wells_mod.filter(regex = 'geom|Web_Station_Name|_r|_c')

In [ ]:
import conda_scripts.arich_functions as af

In [ ]:


m = wells_mod.filter(regex = 'geom|Name|_r|_c').explore( marker_kwds = {'radius':5, 'color':'black'}, name = 'Monitoring Wells Sites Visit Notes')

ibound.explore(m = m, style_kwds = {'weight':1,'fill':False}, name = 'Model Boundary')
swr.explore(m = m, style_kwds = {'weight':3,'fill':True, 'color':'green'}, name = "SWR Cells")
sfr.explore(m = m, style_kwds = {'weight':3,'fill':True, 'color':'grey'}, name = 'SFR Cells')


# wiski_meta.explore( marker_kwds = {'radius':5, 'color':'cyan'}, m = m, name = 'Wiski Wells')
utils.folium_maps.add_layers(m)
m.save(os.path.join(out_folder, 'wells.html'))
m


In [ ]:

def get_ts(idx, datestart):
    ts = hdsobj.get_ts(idx)
    
    df = pd.DataFrame(ts[:,1], columns = [ 'Simulated'])
    
    df = df.set_index(pd.to_datetime(datestart)+ pd.to_timedelta(ts[:,0], unit = 's'))
    
    
    return df

def load_obs(name, datestart = None, end_time = None):
    
    
    fold = r"T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\MonitoringWells"
    
    path = pathlib.Path(fold).joinpath(name.replace(' ','').replace('-','_')+'.csv')
    
    if path.exists():
        print(f"----------\path does exist:\n{path.name}")
        stg = pd.read_csv(path, parse_dates=[0])
        stg = stg.set_index(stg.columns[0])
        stg = stg.resample('1D').mean()
        
        if datestart is not None:
            stg = stg.loc[datestart:,:].iloc[:109]
    
        
    else:
        print(f"path does not exist:\n{path}")
        stg = pd.DataFrame()
    
    return stg
    

In [ ]:
wells_mod

In [ ]:


hds, hdsobj = basic.get_heads(ml)

partics = os.path.join(out_folder,'hydrographs')
for _,  wel in wells_mod.iterrows():
    
    idx = (0, wel.loc['i_r'], wel.loc['j_c'])
    head = get_ts(idx, datestart)
    obs = load_obs(wel.loc['Well Name'], datestart,1)
    
    if obs.shape[0]==0:
        skip_gw_data = False
    else:
        skip_gw_data = True
    
    f = wel['station_no']
    station_name = wel['station_name']
    filename=os.path.join(partics,f'{f}.png')
    # if not os.path.exists(filename):
    nwp = gwp.fancy_plot(station_name,group = None, 
                         filename=os.path.join(partics,f'{f}.png'),
                         allinfo=None,do_regress=False)

    nwp.do_plot(False, skip_gw_data=skip_gw_data, map_buffer_size = 2500, seasonal = False, plot_dry = False, plot_wet = False,
              maptype =  'USA_Topo_Maps')
    


    head.plot(ax = nwp.upleft)
    minya, maxya = head.loc[:,'Simulated'].min(), head.loc[:,'Simulated'].max()
    nwp.upleft.set_ylim([minya-10, maxya+10])
    if obs.shape[0]>1:
        obs.rename(columns = {'Value':'Observed'}).plot(ax = nwp.upleft)
        miny, maxy = obs.loc[:,'Value'].min(), obs.loc[:,'Value'].max()
        nwp.upleft.set_ylim([np.nanmin([miny,minya])-10, np.nanmax([maxy,maxya])+10])
        
    if not skip_gw_data:
        
        nwp.upleft.set_xlim(left = head.index.min()-pd.to_timedelta('1 w'), right = head.index.max()+pd.to_timedelta('1 w'))
        
    nwp.upleft.xaxis.set_major_locator(mdates.AutoDateLocator())
    nwp.upleft.xaxis.set_minor_locator(mdates.DayLocator())
    nwp.upleft.xaxis.set_major_formatter(
                mdates.ConciseDateFormatter(nwp.upleft.xaxis.get_major_locator()))
        
        
    plt.savefig(filename,dpi = 250, bbox_inches ='tight')

    

In [ ]:
# for _,    wel in wells_mod.iterrows():

#     idx = (0, wel.loc['i_r'], wel.loc['j_c'])
#     head = get_ts(idx, datestart)
#     obs = load_obs(wel.loc['Well Name'], datestart,1)
#     ax = head.plot()
#     if obs.shape[0]>1:
#         ax = obs.rename(columns = {'Value':'Observed'}).plot(ax = ax)